### 1. Import and Load Packages 

In [ ]:
# Load packages
import pandas as pd
import numpy as np
from datetime import date
import csv
import re
import datetime
import os
from sklearn.feature_selection import VarianceThreshold
from pandas import read_csv
from datetime import date
import kaplanmeier as km

import datetime
from sklearn.decomposition import PCA
import pylab as pl
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
%matplotlib inline
from qqman import qqman
import pandas as pd
from pandas_plink import read_plink
from scipy.stats import ttest_ind
import math
import os
from lifelines import KaplanMeierFitter
from pandas import DataFrame
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)
import numpy as np
from pandas.plotting import scatter_matrix
from sklearn import preprocessing
from numpy import set_printoptions
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import datasets
import seaborn as sns; sns.set()
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
#from xgboost import XGBClassifier
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
from numpy import arange
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import pandas as pd
from google.cloud import bigquery
import os
import matplotlib.pyplot as plt
import textwrap
import re
from tqdm import tqdm
%matplotlib inline

### 2. Load cohorts from BigQuery 

##### exp_2022 and out_2022

In [ ]:
user_id = 'kbechler'
nero_gcp_project = 'som-nero-nigam-starr'
cdm_project_id = 'som-nero-nigam-starr'
cdm_dataset_id = 'starr_omop_cdm5_deid_20211213'
work_project_id = 'som-nero-nigam-starr'
work_dataset_id = f'{user_id}_explore'
cdm_subset_dataset_id = 'cdm_subset'

Set up connection

In [ ]:
# Use correct path whether you are local or Nero
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/kbechler/adc.json' 

# Set correct Nero project
os.environ['GCLOUD_PROJECT'] = nero_gcp_project
client = bigquery.Client(project = work_project_id)

In [ ]:
sql = "select * from som-nero-nigam-starr.kbechler_explore.exp_2022"
exp_df = client.query(sql).to_dataframe()

sql = "select * from som-nero-nigam-starr.kbechler_explore.out_2022"
out_df = client.query(sql).to_dataframe()

In [ ]:
print("Number of patients in exposure cohort: ", len(exp_df))
print("Number of patients in outcome cohort:", len(out_df))

### 3. Find first instance of LN diagnosis in positive cases

In [ ]:
# Merge with condition df
condition_df = pd.read_csv('/home/kbechler/notebooks/data/condition_occurrence.tsv', sep='\t')

In [ ]:
condition_df.head()

Confirm proper number of patients have LN condition concept id

In [ ]:
ln_condition = condition_df[condition_df.condition_concept_id == 4285717]
print("Number of patients with ln condition concept:", ln_condition.person_id.nunique())

Find minimum date of LN diagnosis per patient

In [ ]:
# Group by unique person id and min condition concept ID for LN
ln_min = ln_condition.groupby('person_id', as_index = False)['condition_start_DATETIME'].min()
ln_min.columns = ['person_id', 'ln_diag_date']
print(len(ln_min))
print(sum(ln_min.ln_diag_date.isna()))
ln_min.head()

### 4. Create patient level dataframe 

In [ ]:
# Merge with exp cohort
patient_df = pd.merge(exp_df, ln_min, left_on = 'subject_id', right_on = 'person_id', how = 'left')

In [ ]:
print(len(patient_df))

In [ ]:
patient_df.head()

In [ ]:
sum(patient_df.ln_diag_date.notnull())

In [ ]:
patient_df.dtypes

In [ ]:
# Normalize diag data
patient_df['cohort_start_date'] = pd.to_datetime(patient_df['cohort_start_date']).dt.normalize()
patient_df['ln_diag_date'] = pd.to_datetime(patient_df['ln_diag_date']).dt.normalize()

In [ ]:
patient_df.head()

Check to see if patients need to be removed who have had LN diag date prior to cohort start date or same as 
cohort start date

In [ ]:
# Calculate difference from first instance of SLE to first instance of LN
patient_df['time_sle_to_ln'] = pd.to_numeric((patient_df.ln_diag_date - patient_df.cohort_start_date).dt.days, 
                                             downcast = "integer")

In [ ]:
patient_df.time_sle_to_ln.describe()

In [ ]:
print(sum(patient_df.time_sle_to_ln.isna()))

In [ ]:
print(sum(patient_df.time_sle_to_ln > 0))

In [ ]:
patient_df.time_sle_to_ln.describe()

### 5. Time from SLE to LN diagnosis  

In [ ]:
os.getcwd()

In [ ]:
# Create bins of size 1
w = 50
n = math.ceil((patient_df.time_sle_to_ln.max() - patient_df.time_sle_to_ln.min())/w)


# Plot histogram of # days to SLE
plt.figure(figsize=(12,7))
sns.displot(patient_df['time_sle_to_ln'], kde = False, bins = n)
plt.title('Time from 1st Instance of SLE to LN Diagnosis', fontsize=18)
plt.xlabel('# of Days', fontsize=16)
plt.ylabel('Frequency', fontsize=16)
plt.xlim(0,1000)
plt.savefig('/home/kbechler/sle_to_ln.png')
plt.show()

In [ ]:
# Create bins of size 1
w = 1
n = math.ceil(((patient_df.time_sle_to_ln/365).max() - (patient_df.time_sle_to_ln/365).min())/w)


# Plot histogram of # days to SLE
plt.figure(figsize=(12,7))
sns.distplot(patient_df['time_sle_to_ln']/365, kde = False, bins = n)
plt.title('Time from SLE Diagnosis to LN Diagnosis', fontsize=18)
plt.xlabel('# of Years', fontsize=16)
plt.ylabel('# Number of Patients', fontsize=16)
#plt.xlim(0,50)
#plt.ylim = (0,1)
plt.savefig('/home/kbechler/sle_to_ln2.png')
plt.show()

In [ ]:
person_df = pd.read_csv('/home/kbechler/notebooks/data/person.tsv', sep='\t')

In [ ]:
len(person_df)

In [ ]:
patient_df_new = pd.merge(patient_df, person_df[['person_id', 'birth_DATETIME']], left_on = 'subject_id', 
                         right_on = 'person_id', how = 'left')

In [ ]:
patient_df_new.head()

In [ ]:
patient_df_new.dtypes

In [ ]:
patient_df_new.birth_DATETIME = pd.to_datetime(patient_df_new.birth_DATETIME).dt.normalize()

In [ ]:
# Subtract birth_DATETIME from cohort_start_date to get age of patient in years
patient_df_new['age'] = pd.to_numeric((patient_df_new.cohort_start_date - patient_df_new.birth_DATETIME).dt.days, 
downcast = 'integer')

In [ ]:
patient_df_new.head()

In [ ]:
plt.figure(figsize=(12,7))
sns.distplot(patient_df_new['age']/365, kde = False)
plt.title('Age at SLE Diagnosis', fontsize=18)
plt.xlabel('# of Years', fontsize=16)
plt.ylabel('# of Patients', fontsize=16)
plt.axvline((patient_df_new['age']/365).mean(), color='k', linestyle='dashed', linewidth=1)
min_ylim, max_ylim = plt.ylim()
plt.text((patient_df_new['age']/365).mean()*1.1, max_ylim*0.9, 'Mean: {:.2f}'.format((patient_df_new['age']/365).mean()))
plt.savefig('/home/kbechler/sle_age.png')
plt.show()

### 6. Create Venn Diagram for SLE Cohort looking at condition vs. ICD vs. notes 

##### Original exposure cohort with sql from ATLAS 

In [ ]:
user_id = 'kbechler'
nero_gcp_project = 'som-nero-nigam-starr'
cdm_project_id = 'som-nero-nigam-starr'
cdm_dataset_id = 'starr_omop_cdm5_deid_20211213'
work_project_id = 'som-nero-nigam-starr'
work_dataset_id = f'{user_id}_explore'
cdm_subset_dataset_id = 'cdm_subset'
cohort_table_id = 'exp_cohort'
cohort_id = 21

In [ ]:
sql = "select * from som-nero-nigam-starr.kbechler_explore.exp_cohort"
atlas_exp_cohort = client.query(sql).to_dataframe()

In [ ]:
import matplotlib_venn as venn
from matplotlib_venn import venn3
from matplotlib_venn import venn2

In [ ]:
# Filter to person_id by condition_occurrence SLE per STARR_OMOP
# Condition codes
# 4055640: Lung disease with systemic lupus erythematosus
# 4344158: Systemic lupus erythematosus with organ/system involvement
# 4285717: SLE glomerulopehritis syndrome
# 4063581: Drug-induced systemic lupus erythematosus
# 4149913: Systemic lupus erythematosus with pericarditis
# 257628: Systemic lupus erythematosus

In [ ]:
# Condition occurrence of SLE
# Mapped and descendants
sle_condition_list = [4055640, 4344158, 4285717 , 4063581, 4149913, 257628]

In [ ]:
# Read in data from condition occurrence table
sql = """SELECT * 
FROM `som-nero-nigam-starr.starr_omop_cdm5_deid_20211213.condition_occurrence`
WHERE condition_concept_id IN (4055640, 4344158, 4285717 , 4063581, 4149913, 257628)"""
sle_condition = client.query(sql).to_dataframe()

In [ ]:
# Count how many times each patient has condition occurrence associated with SLE
sle_condition_count = sle_condition.groupby(['person_id'], as_index = False).agg({
    'condition_occurrence_id': 'count'
})

In [ ]:
# Draw histogram
# Create bins
w = 1
n = math.ceil((sle_condition_count.condition_occurrence_id.max() - sle_condition_count.condition_occurrence_id.min())/w)


# Plot histogram
plt.figure(figsize = (12, 7))
sns.histplot(sle_condition_count['condition_occurrence_id'], kde = False, color = 'c' , bins = n)
plt.title('# SLE Condition Occurrences Per Patient')
plt.xlabel('Number of Condition Occurrence Instances')
plt.ylabel('Frequency')
plt.xlim(0,50)
plt.show()

In [ ]:
print("Number of patients with SLE condition occurrence <= 1:", sum(sle_condition_count.condition_occurrence_id <=1))
print("Number of patients with SLE condition occurrence > 1:", sum(sle_condition_count.condition_occurrence_id >1))

In [ ]:
# Filter to person_id by ICD9/ICD10 diagnoses
# ICD10 = M32.9
# ICD9 = 710.0

# Read data from condition occurrence table
sql = """SELECT * 
FROM `som-nero-nigam-starr.starr_omop_cdm5_deid_20211213.condition_occurrence`
WHERE condition_source_value IN ('M32.9', '710.0')"""
sle_icd = client.query(sql).to_dataframe()

In [ ]:
print("Number of unique patients with ICD occurrence:", len(sle_icd.person_id.unique()))

In [ ]:
# Count number of times ICD code occurs per patient
sle_icd_count = sle_icd.groupby(['person_id'], as_index = False).agg({
    'condition_source_value': 'count'
})

In [ ]:
sle_icd_count.head()

In [ ]:
print("Number of patients with <= 1 ICD SLE mention:", sum(sle_icd_count.condition_source_value <= 1))
print("Number of patients with > 1 ICD SLE mention:", sum(sle_icd_count.condition_source_value > 1))

In [ ]:
# Draw histogram
# Create bins
w = 1
n = math.ceil((sle_icd_count.condition_source_value.max() - sle_icd_count.condition_source_value.min())/w)


# Plot histogram
plt.figure(figsize = (12, 7))
sns.histplot(sle_icd_count['condition_source_value'], kde = False, color = 'c' , bins = n)
plt.title('# SLE ICD9/10 Per Patient')
plt.xlabel('Number of ICD Diag Instances')
plt.ylabel('Frequency')
plt.xlim(0,50)
plt.show()

In [ ]:
# Create Venn Diagram
def venn_diagram(condition, icd, labels = ['condition', 'icd']):
    
    a = list(set(condition))
    b = list(set(icd))

    only_a = len([x for x in a if x not in b])
    only_b = len([x for x in b if x not in a])
    print(only_a)
    print(only_b)
    
    a_b = len(np.intersect1d(a, b))
    print(a_b)
    
    venn2(subsets = (only_a, only_b, a_b), set_labels=labels)

In [ ]:
venn_diagram(sle_condition.person_id,sle_icd.person_id)
plt.savefig('/home/kbechler/VennDiagram2_4_5.png')
plt.show()

In [ ]:
patient_df.head()

In [ ]:
# Build 3 category venn diagram
def venn_diagram(condition, icd, notes, labels = ['SLE Condition Occurrence', 'SLE ICD-9/ICD-9 Diags', 'SLE Clinical Notes']):
    a = list(set(condition))
    b = list(set(icd))
    c = list(set(notes))
    
    only_a = len([x for x in a if x not in b+c])
    only_b = len([x for x in b if x not in a+c])
    only_c = len([x for x in c if x not in a+b])
    
    a_b = len(np.intersect1d(a,b))
    a_c = len(np.intersect1d(a,c))
    b_c = len(np.intersect1d(b,c))
    
    a_b_c = len([x for x in a if (x in b) and (x in c)])
    venn3(subsets = (only_a, only_b, a_b, only_c, a_c, b_c, a_b_c), set_labels=labels)   
    

venn_diagram([1, 2], [1, 2, 3, 4], [1, 2, 3])

In [ ]:
venn_diagram(sle_condition.person_id,sle_icd.person_id, patient_df.subject_id)
plt.savefig('/home/kbechler/VennDiagram_3_31.png')
plt.show()

### 7. Look at patient numbers for multiple prediction models vs. survival model or single prediction model 

In [ ]:
# Multiple predictions fixed interval
# Current patient dataframe
print("Number of SLE patients:", len(patient_df))
print("Number of LN patients:", len(patient_df[patient_df.ln_diag_date.notna()]))

In [ ]:
# Create index 0 date = first instance of SLE diagnosis
patient_df['index_0'] = patient_df['cohort_start_date']

In [ ]:
patient_df['ln_event_0'] = np.where(patient_df['time_sle_to_ln'] <= 365, 1, 0)

In [ ]:
sum(patient_df.ln_event_0)

In [ ]:
from datetime import datetime, timedelta

In [ ]:
# Create index 1 date = 1 year from first instance of SLE diagnosis
patient_df['index_1'] = patient_df['index_0'] + timedelta(days = 365)

In [ ]:
patient_df['ln_event_1'] = np.where((patient_df['time_sle_to_ln'] > 365) & (patient_df['time_sle_to_ln'] <= 730), 1, 0)
patient_df['ln_event_1'] = np.where((patient_df['ln_event_0'] == 1), np.nan, patient_df['ln_event_1'])

In [ ]:
print(patient_df.ln_event_1.sum(skipna = True))
print(len(patient_df[patient_df.ln_event_1 == 0]))
print(sum(patient_df.ln_event_1.isna()))

In [ ]:
patient_df['index_2'] = patient_df['index_1'] + timedelta(days = 365)
patient_df['index_3'] = patient_df['index_2'] + timedelta(days = 365)
patient_df['index_4'] = patient_df['index_3'] + timedelta(days = 365)
patient_df['index_5'] = patient_df['index_4'] + timedelta(days = 365)

In [ ]:
patient_df['index_6'] = patient_df['index_5'] + timedelta(days = 365)
patient_df['index_7'] = patient_df['index_6'] + timedelta(days = 365)
patient_df['index_8'] = patient_df['index_7'] + timedelta(days = 365)
patient_df['index_9'] = patient_df['index_8'] + timedelta(days = 365)

In [ ]:
patient_df['ln_event_2'] = np.where((patient_df['time_sle_to_ln'] > 730) & (patient_df['time_sle_to_ln'] <= 1095), 1, 0)
patient_df['ln_event_2'] = np.where((patient_df['ln_event_1'] == 1) | (patient_df['ln_event_0'] == 1), np.nan,patient_df['ln_event_2'])


patient_df['ln_event_3'] = np.where((patient_df['time_sle_to_ln'] > 1095) & (patient_df['time_sle_to_ln'] <= 1490), 1, 0)
patient_df['ln_event_3'] = np.where((patient_df['ln_event_2'] == 1) |(patient_df['ln_event_1'] == 1) | (patient_df['ln_event_0'] == 1), np.nan,patient_df['ln_event_3'])

patient_df['ln_event_4'] = np.where((patient_df['time_sle_to_ln'] > 1490) & (patient_df['time_sle_to_ln'] <= 1825), 1, 0)
patient_df['ln_event_4'] = np.where((patient_df['ln_event_3'] == 1) |(patient_df['ln_event_2'] == 1) | (patient_df['ln_event_1'] == 1) | (patient_df['ln_event_0'] == 1), np.nan,patient_df['ln_event_4'])

patient_df['ln_event_5'] = np.where((patient_df['time_sle_to_ln'] > 1825) & (patient_df['time_sle_to_ln'] <= 2190), 1, 0)
patient_df['ln_event_5'] = np.where((patient_df['ln_event_4'] == 1) |(patient_df['ln_event_3'] == 1) |(patient_df['ln_event_2'] == 1) | (patient_df['ln_event_1'] == 1) | (patient_df['ln_event_0'] == 1), np.nan,patient_df['ln_event_5'])


patient_df['ln_event_6'] = np.where((patient_df['time_sle_to_ln'] > 2190) & (patient_df['time_sle_to_ln'] <= 2555), 1, 0)
patient_df['ln_event_6'] = np.where((patient_df['ln_event_5'] == 1) |(patient_df['ln_event_4'] == 1) |(patient_df['ln_event_3'] == 1) |(patient_df['ln_event_2'] == 1) | (patient_df['ln_event_1'] == 1) | (patient_df['ln_event_0'] == 1), np.nan,patient_df['ln_event_6'])


patient_df['ln_event_7'] = np.where((patient_df['time_sle_to_ln'] > 2555) & (patient_df['time_sle_to_ln'] <= 2920), 1, 0)
patient_df['ln_event_7'] = np.where((patient_df['ln_event_6'] == 1) |(patient_df['ln_event_5'] == 1) |(patient_df['ln_event_4'] == 1) |(patient_df['ln_event_3'] == 1) |(patient_df['ln_event_2'] == 1) | (patient_df['ln_event_1'] == 1) | (patient_df['ln_event_0'] == 1), np.nan,patient_df['ln_event_7'])


patient_df['ln_event_8'] = np.where((patient_df['time_sle_to_ln'] > 2920) & (patient_df['time_sle_to_ln'] <= 3285), 1, 0)
patient_df['ln_event_8'] = np.where((patient_df['ln_event_7'] == 1) |(patient_df['ln_event_6'] == 1) |(patient_df['ln_event_5'] == 1) |(patient_df['ln_event_4'] == 1) |(patient_df['ln_event_3'] == 1) |(patient_df['ln_event_2'] == 1) | (patient_df['ln_event_1'] == 1) | (patient_df['ln_event_0'] == 1), np.nan,patient_df['ln_event_8'])


patient_df['ln_event_9'] = np.where((patient_df['time_sle_to_ln'] > 3285) & (patient_df['time_sle_to_ln'] <= 3650), 1, 0)
patient_df['ln_event_9'] = np.where((patient_df['ln_event_8'] == 1) |(patient_df['ln_event_7'] == 1) |(patient_df['ln_event_6'] == 1) |(patient_df['ln_event_5'] == 1) |(patient_df['ln_event_4'] == 1) |(patient_df['ln_event_3'] == 1) |(patient_df['ln_event_2'] == 1) | (patient_df['ln_event_1'] == 1) | (patient_df['ln_event_0'] == 1), np.nan,patient_df['ln_event_9'])


patient_df['ln_event_10'] = np.where((patient_df['time_sle_to_ln'] > 3650) & (patient_df['time_sle_to_ln'] <= 4015), 1, 0)
patient_df['ln_event_10'] = np.where((patient_df['ln_event_9'] == 1) |(patient_df['ln_event_8'] == 1) |(patient_df['ln_event_7'] == 1) |(patient_df['ln_event_6'] == 1) |(patient_df['ln_event_5'] == 1) |(patient_df['ln_event_4'] == 1) |(patient_df['ln_event_3'] == 1) |(patient_df['ln_event_2'] == 1) | (patient_df['ln_event_1'] == 1) | (patient_df['ln_event_0'] == 1), np.nan,patient_df['ln_event_10'])


patient_df['ln_event_11'] = np.where((patient_df['time_sle_to_ln'] > 4015) & (patient_df['time_sle_to_ln'] <= 4380), 1, 0)
patient_df['ln_event_11'] = np.where((patient_df['ln_event_10'] == 1) |(patient_df['ln_event_9'] == 1) |(patient_df['ln_event_8'] == 1) |(patient_df['ln_event_7'] == 1) |(patient_df['ln_event_6'] == 1) |(patient_df['ln_event_5'] == 1) |(patient_df['ln_event_4'] == 1) |(patient_df['ln_event_3'] == 1) |(patient_df['ln_event_2'] == 1) | (patient_df['ln_event_1'] == 1) | (patient_df['ln_event_0'] == 1), np.nan,patient_df['ln_event_11'])


patient_df['ln_event_12'] = np.where((patient_df['time_sle_to_ln'] > 4380) & (patient_df['time_sle_to_ln'] <= 4745), 1, 0)
patient_df['ln_event_12'] = np.where((patient_df['ln_event_11'] == 1) |(patient_df['ln_event_10'] == 1) |(patient_df['ln_event_9'] == 1) |(patient_df['ln_event_8'] == 1) |(patient_df['ln_event_7'] == 1) |(patient_df['ln_event_6'] == 1) |(patient_df['ln_event_5'] == 1) |(patient_df['ln_event_4'] == 1) |(patient_df['ln_event_3'] == 1) |(patient_df['ln_event_2'] == 1) | (patient_df['ln_event_1'] == 1) | (patient_df['ln_event_0'] == 1), np.nan,patient_df['ln_event_12'])


In [ ]:
# Create dataframe
one_year_predictions = pd.DataFrame()

In [ ]:
row = (len(patient_df),patient_df.ln_event_0.sum(skipna = True))
row = pd.Series(row)
one_year_predictions = one_year_predictions.append(row, ignore_index = True )

In [ ]:
row = (len(patient_df) - len(patient_df[patient_df.ln_event_0 == 1]),patient_df.ln_event_1.sum(skipna = True))
row = pd.Series(row)
one_year_predictions = one_year_predictions.append(row, ignore_index = True )

row = (len(patient_df) - len(patient_df[patient_df.ln_event_1 == 1]) - len(patient_df[patient_df.ln_event_0 == 1]),patient_df.ln_event_2.sum(skipna = True))
row = pd.Series(row)
one_year_predictions = one_year_predictions.append(row, ignore_index = True )

row = (len(patient_df) - len(patient_df[patient_df.ln_event_2 == 1]) - len(patient_df[patient_df.ln_event_1 == 1]) - len(patient_df[patient_df.ln_event_0 ==1]),patient_df.ln_event_3.sum(skipna = True))
row = pd.Series(row)
one_year_predictions = one_year_predictions.append(row, ignore_index = True )

row = (len(patient_df) - len(patient_df[patient_df.ln_event_3 == 1]) - len(patient_df[patient_df.ln_event_2 ==1])- len(patient_df[patient_df.ln_event_1 == 1]) - len(patient_df[patient_df.ln_event_0 == 1]),patient_df.ln_event_4.sum(skipna = True))
row = pd.Series(row)
one_year_predictions = one_year_predictions.append(row, ignore_index = True )

row = (len(patient_df) - len(patient_df[patient_df.ln_event_4 == 1]) - len(patient_df[patient_df.ln_event_3 == 1]) - len(patient_df[patient_df.ln_event_2 == 1]) - len(patient_df[patient_df.ln_event_1 == 1]) - len(patient_df[patient_df.ln_event_0 == 1]),patient_df.ln_event_5.sum(skipna = True))
row = pd.Series(row)
one_year_predictions = one_year_predictions.append(row, ignore_index = True )

row = (len(patient_df) - len(patient_df[patient_df.ln_event_5 == 1]) - len(patient_df[patient_df.ln_event_4 == 1]) - len(patient_df[patient_df.ln_event_3 ==1]) - len(patient_df[patient_df.ln_event_2 ==1]) - len(patient_df[patient_df.ln_event_1 ==1]) - len(patient_df[patient_df.ln_event_0 == 1]),patient_df.ln_event_6.sum(skipna = True))
row = pd.Series(row)
one_year_predictions = one_year_predictions.append(row, ignore_index = True )

row = (len(patient_df) - len(patient_df[patient_df.ln_event_6 == 1]) - len(patient_df[patient_df.ln_event_5 == 1]) - len(patient_df[patient_df.ln_event_4 == 1]) - len(patient_df[patient_df.ln_event_3 == 1]) - len(patient_df[patient_df.ln_event_2 == 1]) - len(patient_df[patient_df.ln_event_1 == 1]) - len(patient_df[patient_df.ln_event_0 ==1]),patient_df.ln_event_7.sum(skipna = True))
row = pd.Series(row)
one_year_predictions = one_year_predictions.append(row, ignore_index = True )

row = (len(patient_df) - len(patient_df[patient_df.ln_event_7 == 1]) - len(patient_df[patient_df.ln_event_6 ==1]) - len(patient_df[patient_df.ln_event_5 ==1]) - len(patient_df[patient_df.ln_event_4 == 1]) - len(patient_df[patient_df.ln_event_3 == 1]) - len(patient_df[patient_df.ln_event_2 ==1]) - len(patient_df[patient_df.ln_event_1 ==1]) - len(patient_df[patient_df.ln_event_0 ==1]),patient_df.ln_event_8.sum(skipna = True))
row = pd.Series(row)
one_year_predictions = one_year_predictions.append(row, ignore_index = True )

row = (len(patient_df) - len(patient_df[patient_df.ln_event_8 == 1]) - len(patient_df[patient_df.ln_event_7 == 1]) - len(patient_df[patient_df.ln_event_6 == 1])- len(patient_df[patient_df.ln_event_5 == 1]) - len(patient_df[patient_df.ln_event_4 == 1]) - len(patient_df[patient_df.ln_event_3 == 1]) - len(patient_df[patient_df.ln_event_2 == 1]) - len(patient_df[patient_df.ln_event_1 == 1]) - len(patient_df[patient_df.ln_event_0 == 1]),patient_df.ln_event_9.sum(skipna = True))
row = pd.Series(row)
one_year_predictions = one_year_predictions.append(row, ignore_index = True )

row = (len(patient_df) - len(patient_df[patient_df.ln_event_9 == 1]) - len(patient_df[patient_df.ln_event_8 == 1]) - len(patient_df[patient_df.ln_event_7 == 1]) - len(patient_df[patient_df.ln_event_6 == 1]) - len(patient_df[patient_df.ln_event_5 == 1]) - len(patient_df[patient_df.ln_event_4 ==1]) - len(patient_df[patient_df.ln_event_3 == 1]) - len(patient_df[patient_df.ln_event_2 == 1]) - len(patient_df[patient_df.ln_event_1 == 1]) - len(patient_df[patient_df.ln_event_0 ==1]),patient_df.ln_event_10.sum(skipna = True))
row = pd.Series(row)
one_year_predictions = one_year_predictions.append(row, ignore_index = True )

row = (len(patient_df) - len(patient_df[patient_df.ln_event_10 == 1]) - len(patient_df[patient_df.ln_event_9 ==1]) - len(patient_df[patient_df.ln_event_8 ==1]) - len(patient_df[patient_df.ln_event_7 == 1])- len(patient_df[patient_df.ln_event_6 ==1])- len(patient_df[patient_df.ln_event_5 ==1]) - len(patient_df[patient_df.ln_event_4 ==1])- len(patient_df[patient_df.ln_event_3 ==1])- len(patient_df[patient_df.ln_event_2 ==1]) - len(patient_df[patient_df.ln_event_1 ==1]) - len(patient_df[patient_df.ln_event_0 ==1]),patient_df.ln_event_11.sum(skipna = True))
row = pd.Series(row)
one_year_predictions = one_year_predictions.append(row, ignore_index = True )

row = (len(patient_df) - len(patient_df[patient_df.ln_event_11 == 1]) - len(patient_df[patient_df.ln_event_10 ==1]) - len(patient_df[patient_df.ln_event_9 == 1]) - len(patient_df[patient_df.ln_event_8 == 1]) - len(patient_df[patient_df.ln_event_7 ==1]) - len(patient_df[patient_df.ln_event_6 ==1]) - len(patient_df[patient_df.ln_event_5 ==1]) - len(patient_df[patient_df.ln_event_4 ==1]) - len(patient_df[patient_df.ln_event_3 == 1]) - len(patient_df[patient_df.ln_event_2 == 1]) - len(patient_df[patient_df.ln_event_1 ==1]) - len(patient_df[patient_df.ln_event_0 ==1]),patient_df.ln_event_12.sum(skipna = True))
row = pd.Series(row)
one_year_predictions = one_year_predictions.append(row, ignore_index = True )

In [ ]:
one_year_predictions.head()

In [ ]:
one_year_predictions.columns = ['sle_patients', 'ln_event_patients']

In [ ]:
one_year_predictions['ln_cum'] = one_year_predictions.ln_event_patients.cumsum()

In [ ]:
print(sum(patient_df.ln_event_0))
print(patient_df.ln_event_1.sum(skipna = True))
print(patient_df.ln_event_2.sum(skipna = True))
print(patient_df.ln_event_3.sum(skipna = True))
print(patient_df.ln_event_4.sum(skipna = True))
print(patient_df.ln_event_5.sum(skipna = True))
print(patient_df.ln_event_6.sum(skipna = True))
print(patient_df.ln_event_7.sum(skipna = True))
print(patient_df.ln_event_8.sum(skipna = True))
print(patient_df.ln_event_9.sum(skipna = True))
print(patient_df.ln_event_10.sum(skipna = True))
print(patient_df.ln_event_11.sum(skipna = True))
print(patient_df.ln_event_12.sum(skipna = True))

In [ ]:
one_year_predictions

In [ ]:
# Number of patients who will develop LN in 1 year?
# Number of patient who will develop LN in the next 2 years?
# Next 3 years?
# Next 4 years?
# Next 5 years?
patient_df['ln_1year'] = np.where(patient_df['time_sle_to_ln'] <= 365, 1, 0)
patient_df['ln_2years'] = np.where(patient_df['time_sle_to_ln'] <= 730, 1, 0)
patient_df['ln_3years'] = np.where(patient_df['time_sle_to_ln'] <= 1095, 1, 0)
patient_df['ln_4years'] = np.where(patient_df['time_sle_to_ln'] <= 1460, 1, 0)
patient_df['ln_5years'] = np.where(patient_df['time_sle_to_ln'] <= (365*5), 1, 0)
patient_df['ln_6years'] = np.where(patient_df['time_sle_to_ln'] <= (365*6), 1, 0)
patient_df['ln_7years'] = np.where(patient_df['time_sle_to_ln'] <= (365*7), 1, 0)
patient_df['ln_8years'] = np.where(patient_df['time_sle_to_ln'] <= (365*8), 1, 0)
patient_df['ln_9years'] = np.where(patient_df['time_sle_to_ln'] <= (365*9), 1, 0)
patient_df['ln_10years'] = np.where(patient_df['time_sle_to_ln'] <= (365*10), 1, 0)
patient_df['ln_11years'] = np.where(patient_df['time_sle_to_ln'] <= (365*11), 1, 0)
patient_df['ln_12years'] = np.where(patient_df['time_sle_to_ln'] <= (365*12), 1, 0)

In [ ]:
# Create dataframe
multiple_year_predictions = pd.DataFrame()

In [ ]:
row = (len(patient_df),patient_df.ln_1year.sum(skipna = True))
row = pd.Series(row)
multiple_year_predictions = multiple_year_predictions.append(row, ignore_index = True )

In [ ]:
row = (len(patient_df),patient_df.ln_2years.sum(skipna = True))
row = pd.Series(row)
multiple_year_predictions = multiple_year_predictions.append(row, ignore_index = True )

row = (len(patient_df),patient_df.ln_3years.sum(skipna = True))
row = pd.Series(row)
multiple_year_predictions = multiple_year_predictions.append(row, ignore_index = True )

row = (len(patient_df),patient_df.ln_4years.sum(skipna = True))
row = pd.Series(row)
multiple_year_predictions = multiple_year_predictions.append(row, ignore_index = True )

row = (len(patient_df),patient_df.ln_5years.sum(skipna = True))
row = pd.Series(row)
multiple_year_predictions = multiple_year_predictions.append(row, ignore_index = True )

row = (len(patient_df),patient_df.ln_6years.sum(skipna = True))
row = pd.Series(row)
multiple_year_predictions = multiple_year_predictions.append(row, ignore_index = True )

row = (len(patient_df),patient_df.ln_7years.sum(skipna = True))
row = pd.Series(row)
multiple_year_predictions = multiple_year_predictions.append(row, ignore_index = True )


row = (len(patient_df),patient_df.ln_8years.sum(skipna = True))
row = pd.Series(row)
multiple_year_predictions = multiple_year_predictions.append(row, ignore_index = True )

row = (len(patient_df),patient_df.ln_9years.sum(skipna = True))
row = pd.Series(row)
multiple_year_predictions = multiple_year_predictions.append(row, ignore_index = True )

row = (len(patient_df),patient_df.ln_10years.sum(skipna = True))
row = pd.Series(row)
multiple_year_predictions = multiple_year_predictions.append(row, ignore_index = True )

row = (len(patient_df),patient_df.ln_11years.sum(skipna = True))
row = pd.Series(row)
multiple_year_predictions = multiple_year_predictions.append(row, ignore_index = True )

row = (len(patient_df),patient_df.ln_12years.sum(skipna = True))
row = pd.Series(row)
multiple_year_predictions = multiple_year_predictions.append(row, ignore_index = True )


In [ ]:
multiple_year_predictions['year'] = np.arange(start = 1, stop = 13, step = 1)

In [ ]:
multiple_year_predictions.columns = ['sle_patients', 'ln_events', 'year']

In [ ]:
multiple_year_predictions

In [ ]:
# Export dataframes
multiple_year_predictions.to_csv('/home/kbechler/multiple_year_predictions_2022.csv', index = False)
one_year_predictions.to_csv('/home/kbechler/one_year_predictions_2022.csv', index = False)

##### Make plots

In [ ]:
labels = ['index_0', 'index_1', 'index_2', 'index_3', 'index_4', 'index_5', 'index_6', 'index_7', 'index_8', 
         'index_9', 'index_10', 'index_11', 'index_12']

In [ ]:
num_sle_patients = one_year_predictions.sle_patients
num_ln_patients = one_year_predictions.ln_event_patients

In [ ]:
width = 0.35
fig, ax = plt.subplots()

ax.bar(labels, num_ln_patients, width, label = "ln_patients")
ax.bar(labels, num_sle_patients, width, label = "sle_patients", bottom = num_ln_patients)

ax.set_ylabel('Number of Patients')
ax.set_title('Patients per Cohort')
ax.legend()


plt.show()

In [ ]:
labels = ['1year', '2years', '3years', '4years', '5years', '6years', '7years', '8years', '9years', '10years', 
         '11years', '12years']
list(multiple_year_predictions)

num_sle_patients = multiple_year_predictions['sle_patients']
num_ln_patients = multiple_year_predictions['ln_events']

width = 0.35

fig, ax = plt.subplots()

ax.bar(labels, num_ln_patients, width, label = "ln_patients")
ax.bar(labels, num_sle_patients, width, label = "sle_patients", bottom = num_ln_patients)

ax.set_ylabel('Number of Patients')
ax.set_title('Patients per Cohort')
ax.legend()


plt.show()

In [ ]:
patient_df.groupby('ln_event').count()

### 8. Kaplan Meier Curve 

In [ ]:
# Get full observation window
sql = "select * from som-nero-nigam-starr.cdm_subset.observation_period"
observation_period = client.query(sql).to_dataframe()

In [ ]:
len(observation_period)

In [ ]:
# Merge with patient_df
list(observation_period)

In [ ]:
patient_df = pd.merge(patient_df, observation_period[['person_id', 'observation_period_end_DATE']],left_on = 'subject_id',
                     right_on = 'person_id', how = 'inner')

In [ ]:
len(patient_df)

In [ ]:
# Convert to datetime
patient_df.observation_period_end_DATE = pd.to_datetime(patient_df.observation_period_end_DATE)

In [ ]:
patient_df['observation_post_sle_obs'] = pd.to_numeric((patient_df.observation_period_end_DATE - 
                                                        patient_df.cohort_start_date).dt.days, downcast = "integer")

In [ ]:
# Create bins of size 1
w = 1
n = math.ceil((patient_df.observation_post_sle_obs.max() - patient_df.observation_post_sle_obs.min())/w)


# Plot histogram of # days to SLE
plt.figure(figsize=(12,7))
sns.displot(patient_df['observation_post_sle_obs'], kde = False, bins = n)
plt.title('Observation Time Post SLE', fontsize=18)
plt.xlabel('# of Days', fontsize=16)
plt.ylabel('# of Patients', fontsize=16)
#plt.xlim(0,)
plt.savefig('/home/kbechler/sle_to_end_obs.png')
plt.show()

In [ ]:
# Create "event_date"
# If patient doesn't have ln diag, event date is last observation date of patient
patient_df['event_date'] = np.where(patient_df.time_sle_to_ln.isna(), patient_df.observation_post_sle_obs,
                                   patient_df.time_sle_to_ln)

In [ ]:
patient_df.event_date.describe()

In [ ]:
kmf = KaplanMeierFitter()

In [ ]:
patient_df['ln_event'] = np.where(patient_df.time_sle_to_ln.isna(), 0, 1)

In [ ]:
kmf.fit(durations = patient_df['event_date'], event_observed = patient_df['ln_event'])

In [ ]:
kmf.event_table

In [ ]:
# Plot the graph
kmf.plot()
plt.title("Kaplan-Meier Estimate")
plt.ylabel("Probability patient not diagnosed with LN")
plt.savefig('/home/kbechler/km_test.png')
plt.show()
print("The median survival time: " ,kmf.median_survival_time_)

In [ ]:
kmf.plot_cumulative_density()

### 9. Export patient dataframe for labels with patients to get LN in x years 

In [ ]:
 patient_df.head()

In [ ]:
patient_df.to_csv('/home/kbechler/patient_df_labels_2022.csv', index = False)